# Munging data with Pandas 


Data "munging" is the series of actions that we take to transform data that we are given from its raw form to something that we can process and understand. 

In neuroimaging, we can roughly divide this process into two steps: in the first, we work with the images that were collected in the scanner. This entails some image processing, some time-series analysis and so forth. In the second step, we might already be looking at features that have been extracted from the data and are abstracted away from the original images. For example, the size of each of a collection of brain regions in each of the subjects in a sample. The second kind of data can be organized into a *tabular* format. We will talk about desirable characteristics for this kind of data in a little bit, but let's start with a really simple example. Consider a dataset describing some fruit:

In [ ]:
fruit = {"UID": ["a101", "o101", "o102", "b101"],
         "Kind": ["Apple", "Orange", "Orange", "Banana"], 
         "Price": [0.1, 0.75, 0.84, 1.2],
         "Weight": [200, 250, 280, 100],
         "Deliciousness": [8, 5, 6, 10]}

As a dictionary, these data would be hard to manipulate. For example, try writing code to display the average price of oranges. An altetnative would have been to store all this data as a Numpy array, but we also can't organize this very well into a numpy array, because it has mixed types and one dimension (the order of items in each list) means one thing, while the other (the different lists) means another thing. 

Pandas (sure, it's an animal, but it also stands for "panel data", which is another name for tabular data) is a software library that helps do exactly these kinds of things with data like this. Over the years, it's become a real "industry standard" for work with these kinds of data and has become a dependency or interoperates well with many other software libraries that analyze, visualize and create such data. Including many neuroscience-specific software. You will see some of these examples here, and once you learn about Pandas, will start noticing this everywhere.

Let's start by initializing a DataFrame, which is the workhorse object for Pandas data manipulations.

In [ ]:
import pandas as pd

In [ ]:
fruit_table = pd.DataFrame(fruit)

In Jupyter, we can show the data in nice tabular format: 

In [ ]:
fruit_table

We can also use the DataFrame to query the data, either by column: 

In [ ]:
fruit_table["Kind"]

Or by designating one of our columns as an index:

In [ ]:
fruit_table = fruit_table.set_index("UID")

In [ ]:
fruit_table

and using that to query the rows: 

In [ ]:
fruit_table.loc["a101"]

You can also combine these to query on rows and columns 

In [ ]:
fruit_table.loc["a101", "Price"]

And even slice on each dimension to get sub-tables:

In [ ]:
fruit_table.loc["a101":"o102", "Price":"Weight"]

There is another kind of indexer, `iloc` that will also give you a specific row based on its serial position in the table:

In [ ]:
fruit_table.iloc[0]

But maybe you can already come up with some ideas about why using this kind of indexing can be a bad idea?

## Computing with DataFrame objects

Much like Numpy arrays, DataFrames have methods attached to them that let us do computations on the values stored in them. However, in contrast to Numpy arrays, averaging is typically (and per default) done column by column (because it wouldn't make sense to average price and weight, for example) and the DataFrame objects are aware of the fact that some of their columns may contain information that cannot be averaged, because it is not numeric.

In [ ]:
fruit_table.mean(numeric_only=True)

We'll see other types of computations a bit further below, but let's 

## Selecting items in a DataFrame

One of the operations that we often want to do with data is to filter it or select based on a particular attribute. For example, we mentioned the idea of computing the average price of oranges in this dataset.

Let's parse this idea bit by bit. First, we'd like to find all the oranges in the `"Kind"` column. When we select a single column from the data, the return value is no longer a `DataFrame`. Instead, it's something that Pandas calls a `Series`

In [ ]:
fruit_kinds = fruit_table["Kind"]

In [ ]:
type(fruit_kinds)

This kind of objects lends itself to Boolean operations. For example, comparisons:

In [ ]:
is_orange = fruit_kinds == "Orange"

The result of this computation is another Pandas `Series` object, but this one has a Boolean datatype. 

In [ ]:
is_orange

In Numpy, we use Boolean arrays to index and select from other arrays of the same shape. In Pandas, we use Boolean Series to index and select on the rows of Pandas DataFrame objects. For example, the following gives us a table with only the Oranges from the original `fruit_table` DataFrame

In [ ]:
fruit_table[is_orange]

We can also do this in one fell swoop, without creating the intermediate objects, and you will often see code that looks like this: 

In [ ]:
orange_table = fruit_table[fruit_table["Kind"] == "Orange"]

 We can proceed to calculate the average price of oranges from the new table:

In [ ]:
orange_table["Price"].mean()

Or even in one call from the original table:

In [ ]:
fruit_table[fruit_table["Kind"] == "Orange"]["Price"].mean()

### Computing with Series

Because each Series does have a data type associated with it, you can do pretty sensible computations with it, such as multiplication by a scalar. For example, if we know that the "Weight" column is given in grams, we can convert all of the weights to Kilogram:

In [ ]:
weight_in_kg = fruit_table["Weight"] / 1000

In [ ]:
weight_in_kg

Interestingly, we can make that into a new column in the existing table: 

In [ ]:
fruit_table["Weight_kg"] = fruit_table["Weight"] / 1000

In [ ]:
fruit_table

And because they all have the same number of items, you can also do item-by-item computations between Series objects: 

In [ ]:
fruit_table["Price_per_kg"] = fruit_table["Price"] / fruit_table["Weight_kg"]

In [ ]:
fruit_table

### Exercise: 

How would you compute a "deliciousness per dollar" index for these fruit?


### Accessing data stored in files 

One of Pandas real super powers is the ability to read data from multiple different sources. 

So long as the data is stored in a way that can be interpreted as a table, Pandas can probably read it. This includes all kinds of proprietary formats (such as xls and stata files), and also all kinds of binary formats (if you like that kind of thing, check out the parquet file format for some high-efficiency ways of storing really large tables). 

Here, we'll look at some data stored in standard, text-based formats. The data we will look at comes from the ABIDE2 dataset and was extracted for this paper: 

> Bethlehem, R.A.I., Seidlitz, J., Romero-Garcia, R. et al. A normative modelling approach reveals age-atypical cortical thickness in a subgroup of males with autism spectrum disorder. Commun Biol 3, 486 (2020). https://doi.org/10.1038/s42003-020-01212-9

ABIDE2 is a large data collection and sharing effort focused on measurements from individuals with autism and control subjects. Let's see what we get when we read the tsv file that contains ABIDE data:

In [ ]:
features = pd.read_csv("/home/jovyan/shared/abide2/abide2.tsv", sep="\t")

In [ ]:
features.shape

In [ ]:
features

The table contains 1004 rows, corresponding to the 1004 subjects in the dataset. 

The six first columns contain information about each subject: a unique identifier, their age, age residuals relative to the site from which their data was collected (which we will not use here, but will come in handy later in the week, when we use it in the machine learning session), their sex, and whether they were diagnosed with autism (where 1 indicates an autism diagnosis and 2 indicates a control subject). 

The remaining 1440 columns contain structural brain features computed with Freesurfer: surface area (fsArea), volume (fsVol), cortical thickness (fsCT), and local gyrification (fsLGI). We have 360 * 4 features, for each of the 360 regions-of-interest in the Human Connectome Project’s multi-modal parcellation.

When data are this big, it's sometimes useful to get a summary of the distributions of each column:

In [ ]:
features.describe()

In this case, we get some interesting information primarily about the age of the subjects. We also get a sense of the distribution among the two classes in `group` (is this data balanced?), but there are still a lot of columns to look at. Without worry about that too much for now, let's proceed to demonstrate some additional useful data management patterns accessible through Pandas.

## Split-apply-combine

This pattern is very common and very useful. The idea is that we can split the data based on some feature, apply some kind of computation to each group and then combine all of the results into a new dataset. 

For example, a pretty obvious thing to do here would be to split or group the data by the `group` variable, and compute some statistics separately for each one of the groups. We could do this based on what we've seen so far -- logical indexing that selects one group after the other and does the operations we want to do on each group separately. But let's introduce a new way of doing this and some new functionality. 

### Groupby

Each DataFrame object has a method `groupby`, which splits the data up internally according to the unique values of a column, or combination of columns. For example, we can ask the DataFrame to give us groups based on the `group` column:

In [ ]:
autism_groups = features.groupby("group")

The resulting object is not a DataFrame! Instead, it's an object that is a little bit inscrutable. 

In [ ]:
autism_groups

To make it more sensible, we have to keep going for a bit more. We've split the data. Next, we can apply a computation and combine back into a new dataset

In [ ]:
autism_groups.mean()

This result is a DataFrame! And now, instead of individuals on each row, it has reduced the data down to means across each group. 

What if we want to explore more complex combinations? For example, all possible combinations of `group` and `sex`? This would already be pretty cumbersome using Series indexing that we saw above. But `groupby` makes it relatively straightforward:
 

In [ ]:
autism_sex_groups = features.groupby(["group", "sex"])

In [ ]:
autism_sex_means = autism_sex_groups.mean()

In [ ]:
autism_sex_means

The new object we get as a result of this computation is a DataFrame, but instead of one index that we can use to select from it, it has multiple levels of indexing (This is called a `MultiIndex`). The first one selects from the `group`:

In [ ]:
autism_sex_means.loc[1]

And the second one selects from among the options in `sex`, so that we can (for example) look at sex differences separately in each `group`:

In [ ]:
autism_sex_means.loc[1, 1] - autism_sex_means.loc[1, 2]

In [ ]:
autism_sex_means.loc[2, 1] - autism_sex_means.loc[2, 2]

## Combining data

Another functionality that Pandas implements that is very useful is the ability to merge different data based on shared variables. For example, the Freesurfer features that we saw above are taken from anatomical scans, and ABIDE also provides some quality information about these anatomical scans: 

In [ ]:
anat_qc = pd.read_csv("/home/jovyan/shared/abide2/abide2_anat_qap.csv")

Looking at the statistics of these QC data is probably pretty useful

In [ ]:
anat_qc.info()

We notice that there is a lot more data here. This might be because some subjects have more than one entry (for example, attended more than one session). We can cut that down by dropping duplicates on the subject ID column. In most cases, we'd need to verify that this command really is doing the right 

In [ ]:
anat_qc = anat_qc.drop_duplicates("Sub_ID")

Now let's describe the data

In [ ]:
anat_qc.describe()

There's still a bit more data here than in the Freesurfer table, and this is something we can explore in a bit more detail. For now, let's move on. This is probably a good time to mention that we can directly visualize data from a DataFrame's `hist` method. We'll look at other visualization methods that take advantage of Pandas later in the week. 

In [ ]:
anat_qc[["SNR", "CNR"]].hist()

Let's say that we'd like to look at the Freesurfer information, but only for subjects that have an SNR larger than 10. One way to do that is to combine the data from the two tables. 

Before we do that, let's just note that there are few different ways that you could combine tables.

We'll go back to much simplified tables, before coming back to do this with the ABIDE data. Imagine we have some more fruit:

In [ ]:
more_fruit = {"UID": ["a102", "a103", "o103", "b102"],
              "Kind": ["Apple", "Apple", "Orange", "Banana"], 
              "Price": [0.09, 0.125, 0.66, 3.0],
              "Weight": [180, 250, 220, 250],
              "Deliciousness": [3, 7, 10, 5]}

another_fruit_table = pd.DataFrame(more_fruit)
another_fruit_table = another_fruit_table.set_index("UID")

One way to combine these is to concatenate the tables. This can be useful for cases where more measurements are incrementally added to a dataset

In [ ]:
pd.concat([fruit_table, another_fruit_table])

Notice that Pandas is not too confused by the fact that these two tables have different numbers of columns, but it also doesn't automatically recompute the values for the table that doesn't have these added columns. 

This kind of combination would not be particularly useful for the ABIDE situation, is it? What do you think would happen if you concatenated the `features` and `anat_qc` tables in this way?

A more reasonable thing to do in this case is to *merge* the data. This is because the In a merge, we designate the variable that is used to merge (in databases, this is sometimes known as a "foreign key"). Here, we can rely on the fact that both tables have subject identifiers. 

In [ ]:
merged = pd.merge(features, anat_qc, left_on="subject", right_on="Sub_ID")

The resulting table has information from both of the source tables. Note that `pd.merge` has different ways of merging data from different sources. What kinds of variations do you think we need to have for this operation? The `how` key-word argument of the merge function is a way to control different ways of doing this merge. Per default, it takes the strategy called an "inner" merge, which preserves only the cases where there is congruence between the keys. This also implies that we'll have the number of subjects of the smaller of the two (1004), dropping those subjects that appear in one table but not the other. This is the strategy that makes sense here, but sometimes you want to keep going with the subjects for whom you don't have data in some columns. In which case, another strategy might make more sense.

In [ ]:
merged.describe()

Now, we can conduct the comparisons we want (e.g., grouping by `sex` and `group`) directly on the merged data, incorporating information about SNR as a selection variable. Putting together much of what we saw above, we can call: 

In [ ]:
merged[merged["SNR"] > 10].groupby(["group", "sex"]).mean()

## Tidy data

A few more words before we wrap up. We saw here operations that we can do with input data using Pandas. This is all pretty elegant, and there's a lot more. But underlying our ability to perform these operations is a secret. The data have to show up looking a very particular way for all this to work. This way is sometimes called "Tidy data" after a concept described by the illustrious computational statistician and statistical software pioneer Hadley Wickham. In a paper called simply ["Tidy data"](https://vita.had.co.nz/papers/tidy-data.pdf) he describes the details of this idea, but it boils down to the idea that each row is an observation in our dataset and each column should be a variable. So, for example, a datset that has a column named "Subject_001" should raise some red flags. This is also something that database people call "Third normal form". There are some variations on this (we'll see a bit of these variations in the visualization tutorial), and some nuances (for example, a column called "is_male" or "is_autism" is much more useful than "sex" and "group"), but that's the big picture idea. Once everything is nice and tidy, Pandas can do a lot with it. 